In [4]:
# Imports
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

ems = pd.read_csv('../../datasets/totalEmissionsPerCountry2000-2020.csv')
egy = pd.read_csv('../../datasets/globalDataOnSustainableEnergy2000-2020.csv')


In [13]:
## Preprocessing

# Filter the dataset
filtered_ems = ems[(ems['Item'] == 'All sectors with LULUCF') & (ems['Element'] == 'Emissions (CO2)')]

# Select the required columns
filtered_ems = filtered_ems[['Area', 'Item', '2000', '2005', '2010', '2015', '2020']]

# Write to a new dataset
filtered_ems.to_csv('../../datasets/graphB/emissionsYesLULUCF.csv', index=False)


In [14]:
# Filter the dataset 2
filtered_ems2 = ems[(ems['Item'] == 'All sectors without LULUCF') & (ems['Element'] == 'Emissions (CO2)')]

# Select the required columns 2
filtered_ems2 = filtered_ems2[['Area', 'Item', '2000', '2005', '2010', '2015', '2020']]

# Write to a new dataset 2
filtered_ems2.to_csv('../../datasets/graphB/emissionsNoLULUCF', index=False)

In [16]:
# Get unique values of 'Entity' and 'Land Area(Km2)'
unique_values = ems[['Entity', 'Land Area(Km2)']].drop_duplicates()

# Write to a new dataset
unique_values.to_csv('../../datasets/graphB/landAreas.csv', index=False)